# Business Logic
---
This is going to be an experimentation for natural language processing in the context of finance. Using libraries like FinBERT, and LDA, we shall leverage language models to help us inform people about what's going in the world of a specific company. Practices will be followed by extracting the article for a given stock and reusing 
the methodddology for analysis here. 
---


# Importing Libraries